<p style="font-size:25px; color:black;"><u><i><b>Predicting the number of customers likely to visit different departments in a store</b></i></u></p>
<p style="font-size:16px; color:#117d30;">
    Time series forecasting is the use of a model to predict future values based on previously observed values.
The AutoML feature of AzureSynapse, in this case uses more than 25 time series forecasting machine learning algorithms to predicts how many customers are likely to visit different departments in a store.
</p>
Note:
</p>
<p style="font-size:15px; color:#117d30;">
 This notebook is written in Scala, and there is interoperability between Scala and Python code.
</p>

<p style="font-size:15px; color:#117d30;">
    <u> Abstract: </u>
</p>
<p style="font-size:16px; color:#117d30;">
1) Ingest  data from Azure Synapse Data Storage account using PySpark.
</p>
<p style="font-size:16px; color:#117d30;">
2) Exploratory Data Analysis 
</p>
<p style="font-size:15px; color:#117d30;">
3) Training more than 25 time series forecasting machine learning algorithms.
</p>
<p style="font-size:15px; color:#117d30;">
4) Predict the number of customers likely to visit different departments in a store by choosing the best performing Machine Learning Algorithm..
</p>


## Introduction
<p style="font-size:16px; color:#117d30;">


### In this notebook we showcased how to:
<p style="font-size:16px; color:#117d30;">
1. Create an experiment using an existing workspace

<p style="font-size:16px; color:#117d30;">
2. Configure AutoML using 'AutoMLConfig'

<p style="font-size:16px; color:#117d30;">
3. Train the model 

<p style="font-size:16px; color:#117d30;">
4. Explore the engineered features and results

<p style="font-size:16px; color:#117d30;">
5. Configuration and remote run of AutoML for a time-series model with lag and rolling window features

<p style="font-size:16px; color:#117d30;">
6. Run and explore the forecast

<p style="font-size:16px; color:#117d30;">
7. Register the model





### Importing required libraries such as azureml, pandas, pandasql, pyspark, and other supporting libraries.



## Cell title


In [3]:
%%pyspark
from azureml.train.automl import AutoMLConfig
# from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from sklearn.metrics import mean_squared_error
import math
from pyspark.sql.window import Window
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.webservice import Webservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

import pandas as pd 
import datetime
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns 
import azureml.train.automl.runtime
import logging
import os, tempfile
import pandas as pd 
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkContext
os.environ['AZURE_SERVICE']="Microsoft.ProjectArcadia"

## *Connecting to Azure Synapse Data Warehouse*
<p style="font-size:16px; color:#117d30;">
    Connection to Azure Synapse Data Warehouse is initiated and the required data is ingested for processing.
    The warehouse is connected with a single line of code. Just point to actions in a table, click on a new notebook, and then click on "Load to DataFrame".  </p>
   <p style="font-size:16px; color:#117d30;"> After providing the necessary details,  the required data is loaded in the form of a Spark dataframe.
One magical line of code converts a dataframe from Scala to Python!
</p>


In [4]:
%%spark
val df = spark.read.sqlanalytics("SQLPool01.dbo.department_visit_customer")
//Create a Temp view for using the dataframe from Scala to Python
  df.createOrReplaceTempView("df")

df: org.apache.spark.sql.DataFrame = [Date: string, Accessories_count: string ... 6 more fields]

In [5]:
%%spark
display(df)

# Exploratory Data Analysis

<p style="font-size:16px; color:#117d30;">
The goal of performing exploratory data analysis is to understand the underlying patterns and correlations among features in the data. 


In [6]:
%%pyspark
#Calling the dataframe df created in Scala to Python
df = sqlContext.table("df")
# *********************
department_visit_data = df.select("*").toPandas()
department_visit_data['Date'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%m/%d/%y')
department_visit_data['Month'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%m')
department_visit_data['DayOfMonth'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%d')
department_visit_data['Year'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%y')
department_visit_data['DayOfWeek'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%a')
department_visit_data[['Accessories_count','Entertainment_count','Gaming','Kids','Mens','Phone_and_GPS','Womens']] = department_visit_data[['Accessories_count','Entertainment_count','Gaming','Kids','Mens','Phone_and_GPS','Womens']].apply(pd.to_numeric)

#display(department_visit_data)

/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead
  warnings.warn(msg, FutureWarning)

In [7]:
#display(department_visit_data)
department_visit_data

Date  Accessories_count    ...      Year  DayOfWeek
0      07/21/19                114    ...        19        Sun
1      11/04/16                309    ...        16        Fri
2      12/14/17                238    ...        17        Thu
3      04/04/16                286    ...        16        Mon
4      10/20/18                353    ...        18        Sat
5      05/31/17                233    ...        17        Wed
6      03/25/18                271    ...        18        Sun
7      01/01/20                284    ...        20        Wed
8      09/05/16                352    ...        16        Mon
9      10/03/15                339    ...        15        Sat
10     08/24/20                233    ...        20        Mon
11     07/29/15                494    ...        15        Wed
12     06/12/15                395    ...        15        Fri
13     11/13/15                 72    ...        15        Fri
14     03/15/20                264    ...        20        Sun
15 

##  Deriving insights from customer visits data  

<p style="font-size:16px; color:#117d30;">
1. Heat Map: Thickness of the color indicates the no of customers visiting the section on that particular day. It provides a quick representation of distribution of traffic across days and in various departments. From the graph, we can infer that more number of customers visit the Entertainment department on Wednesdays, Thursdays and Fridays and there is less foot traffic on Mondays and Fridays in the Phone_and_gps department.


In [8]:
%%pyspark
df_dow = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Mens']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Mens'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Mens']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Womens'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Womens']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Kids'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Kids']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Gaming'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Gaming']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Entertainment'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Entertainment_count']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Accessories'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Accessories_count']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))

df_dow['Phone_and_GPS'] = pd.DataFrame(department_visit_data.groupby('DayOfWeek')[['Phone_and_GPS']].sum().sort_values(by = 'DayOfWeek', 
                                                                 ascending=True))


df_dow.head(10)


Mens  Womens      ...        Accessories  Phone_and_GPS
DayOfWeek                      ...                                  
Fri        140344  140177      ...             856395         145058
Mon        138747  137086      ...             833021         139095
Sat        141426  140095      ...             846022         143285
Sun        135679  135051      ...             810272         135514
Thu        139177  141552      ...             846595         139471
Tue        140676  144406      ...             854834         140383
Wed        149323  148220      ...             888928         147400

[7 rows x 7 columns]

In [9]:
#import pkg_resources
#pkg_resources.get_distribution("seaborn").version
#it should print version '0.10.1'

In [10]:
sns.set()
plt.rcParams['font.size'] = 12
bg_color = (0.88,0.85,0.95)
plt.rcParams['figure.facecolor'] = bg_color
plt.rcParams['axes.facecolor'] = bg_color
fig, ax = plt.subplots(1)
cmap = sns.diverging_palette(10, 150, n=2, as_cmap=True)
#cmap = sns.color_palette("hls", 150)

p = sns.heatmap(df_dow,
                cmap=cmap,
                annot=True,
                fmt="d",
                annot_kws={'size':8},
                ax=ax)
plt.xlabel('Category')
plt.ylabel('Day Of Week')
ax.set_ylim((0,7))
plt.text(5,7.4, "Heat Map", fontsize = 25, color='Black', fontstyle='italic')
 
plt.show()

# Data Manipulation  
<p style="font-size:16px; color:#117d30;">
1. Converting date to a specific format and making date fields relevant for prediction.

<p style="font-size:16px; color:#117d30;">
2. Converting the data type of the columns to numeric before being passed as input to the model.


In [11]:
%%pyspark
department_visit_data = df.select("*").toPandas()
department_visit_data['Date'] = pd.to_datetime(department_visit_data['Date']).dt.strftime('%Y-%m-%d')

department_visit_data[['Accessories_count','Entertainment_count','Gaming','Kids','Mens','Phone_and_GPS','Womens']] = department_visit_data[['Accessories_count','Entertainment_count','Gaming','Kids','Mens','Phone_and_GPS','Womens']].apply(pd.to_numeric)

grouped_data = department_visit_data.groupby('Date', as_index=False).sum()


total_rows = grouped_data.count
print(total_rows)

<bound method DataFrame.count of             Date  Accessories_count   ...    Phone_and_GPS  Womens
0     2014-12-31                406   ...               88      76
1     2015-01-01               1749   ...              246     192
2     2015-01-02               3371   ...              886     476
3     2015-01-03               3254   ...              454     712
4     2015-01-04               3624   ...              791     630
5     2015-01-05               1165   ...              202     187
6     2015-01-06               2387   ...              312     338
7     2015-01-07               2480   ...              314     417
8     2015-01-08               2527   ...              520     430
9     2015-01-09               3646   ...              656     577
10    2015-01-10               2045   ...              406     178
11    2015-01-11               3694   ...              419     580
12    2015-01-12               3539   ...              604     503
13    2015-01-13             

In [12]:
%%pyspark
accessories_data = grouped_data[['Date','Accessories_count']]
accessories_data

Date  Accessories_count
0     2014-12-31                406
1     2015-01-01               1749
2     2015-01-02               3371
3     2015-01-03               3254
4     2015-01-04               3624
5     2015-01-05               1165
6     2015-01-06               2387
7     2015-01-07               2480
8     2015-01-08               2527
9     2015-01-09               3646
10    2015-01-10               2045
11    2015-01-11               3694
12    2015-01-12               3539
13    2015-01-13               3172
14    2015-01-14               4339
15    2015-01-15               2428
16    2015-01-16               3738
17    2015-01-17               4141
18    2015-01-18               4229
19    2015-01-19                307
20    2015-01-20               3622
21    2015-01-21               4330
22    2015-01-22               2282
23    2015-01-23               4813
24    2015-01-24               2220
25    2015-01-25               2000
26    2015-01-26               1677
27  

In [13]:
%%pyspark
total_rows = accessories_data.count
print(total_rows)

<bound method DataFrame.count of             Date  Accessories_count
0     2014-12-31                406
1     2015-01-01               1749
2     2015-01-02               3371
3     2015-01-03               3254
4     2015-01-04               3624
5     2015-01-05               1165
6     2015-01-06               2387
7     2015-01-07               2480
8     2015-01-08               2527
9     2015-01-09               3646
10    2015-01-10               2045
11    2015-01-11               3694
12    2015-01-12               3539
13    2015-01-13               3172
14    2015-01-14               4339
15    2015-01-15               2428
16    2015-01-16               3738
17    2015-01-17               4141
18    2015-01-18               4229
19    2015-01-19                307
20    2015-01-20               3622
21    2015-01-21               4330
22    2015-01-22               2282
23    2015-01-23               4813
24    2015-01-24               2220
25    2015-01-25               

## Split data into train and test set


In [14]:
%%pyspark
train_data = pd.DataFrame()
test_data = pd.DataFrame()

if accessories_data.shape[0] > 55: # len(df) > 10 would also work
    train_data = accessories_data[:55]
    test_data = accessories_data[55:]

In [15]:
%%pyspark
test_data

Date  Accessories_count
55    2015-02-24               4764
56    2015-02-25               2008
57    2015-02-26               3025
58    2015-02-27               3079
59    2015-02-28               2467
60    2015-03-01               3723
61    2015-03-02               2363
62    2015-03-03               2817
63    2015-03-04               3890
64    2015-03-05               3461
65    2015-03-06               2611
66    2015-03-07               3249
67    2015-03-08               1921
68    2015-03-09               6007
69    2015-03-10               3343
70    2015-03-11               2446
71    2015-03-12               2806
72    2015-03-13               1910
73    2015-03-14               1834
74    2015-03-15               1716
75    2015-03-16               1427
76    2015-03-17               3392
77    2015-03-18               3483
78    2015-03-19               3331
79    2015-03-20               2199
80    2015-03-21                768
81    2015-03-22               3658
82  

## Train

<p style="font-size:16px; color:#117d30;">
1. Instantiate an AutoMLConfig object. 
<p style="font-size:16px; color:#117d30;">
2. The configuration below defines the settings and data used to run the experiment. 


## Set AutoML Configuration Parameters

<p style="font-size:16px; color:#117d30;">
    The forecast horizon is the number of periods into the future that the model should predict. 

<p style="font-size:16px; color:#117d30;">
    It is generally recommended that users set forecast horizons to less than 100 time periods

<p style="font-size:16px; color:#117d30;">
    Furthermore, AutoML's memory use and computation time increases in proportion to the length of the horizon, so consider carefully how this value is set. 

<p style="font-size:16px; color:#117d30;">
    If a long horizon forecast really is necessary, consider aggregating the series to a coarser time scale.


In [16]:
%%pyspark
automl_settings = {
   'time_column_name':'Date',
   'max_horizon': 25
}

In [17]:
%%pyspark
automl_config = AutoMLConfig( 
                            #forecasting for time-series tasks
                            task='forecasting',
                            #measuere for evaluating the performance of the models
                            primary_metric='normalized_root_mean_squared_error',
                            #Maximum amount of time in minutes that the experiment take before it terminates.
                            experiment_timeout_minutes=15,
                            enable_early_stopping=True,
                            training_data=train_data,
                            label_column_name='Accessories_count',
                            #Rolling Origin Validation is used to split time-series in a temporally consistent way.
                            n_cross_validations=4,
                            # Flag to enble early termination if the score is not improving in the short term.
                            enable_ensembling=False,
                            verbosity=logging.INFO,
                            **automl_settings)

In [18]:
%%pyspark
subscription_id='#SUBSCRIPTION_ID#'
resource_group='Synapse-WWI-Lab'
workspace_name='#AML_WORKSPACE_NAME#'
ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
ws.write_config()
ws = Workspace.from_config()
experiment = Experiment(ws, "Department_Visit_Count_20thFeb_3")

## Run The Experiment
<p style="font-size:16px; color:#117d30;">
Automated ML runs more than 25 Machine Learning Algorithms and grades them according to performance.


In [19]:
%%pyspark
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_07e5388c-f6d0-4e14-9bfc-68156dc1ff3f

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.

***********************************

## Retrieve the Best Model


In [20]:
%%pyspark
best_run, fitted_model = local_run.get_output()

In [21]:
%%pyspark
print(best_run)

Run(Experiment: Department_Visit_Count_20thFeb_3,
Id: AutoML_07e5388c-f6d0-4e14-9bfc-68156dc1ff3f_6,
Type: None,
Status: Completed)

In [22]:
%%pyspark
print(fitted_model)

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
     steps=[('timeseriestransformer', TimeSeriesTransformer(featurization_config=None, logger=None,
           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)), ('StandardScalerWrapper', <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fc0ec21c438>), ('RandomForestRegresso...timators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
              stddev=None)

# Evaluate the Model Performance
<p style="font-size:16px; color:#117d30;">Here we have used Root Mean Squared Error (RMSE) for evaluation.</p>


In [23]:
%%pyspark
test_labels = test_data.pop("Accessories_count").values
predict_labels = fitted_model.predict(test_data)
actual_labels = test_labels.flatten()

In [24]:
%%pyspark
rmse = math.sqrt(mean_squared_error(actual_labels,predict_labels))
rmse

1093.0785912945564

In [25]:
%%pyspark
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(actual_labels,predict_labels):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.31736684838093515

Model Accuracy:
0.6826331516190649

In [26]:
%%pyspark
predicted_value = fitted_model.predict(test_data)
actual_value = test_labels.flatten() 
actual_value = actual_value.tolist()
predicted_value = predicted_value.tolist()
output_df= pd.DataFrame({'actual_value':actual_value,'predicted_value':predicted_value})
output_df['Error_Rate %'] = 100*((output_df['actual_value']-output_df['predicted_value'])/(output_df['actual_value']))

output_df

actual_value  predicted_value  Error_Rate %
0             4764          2402.20         49.58
1             2008          2578.56        -28.41
2             3025          2589.04         14.41
3             3079          2430.48         21.06
4             2467          2324.44          5.78
5             3723          2484.52         33.27
6             2363          2378.16         -0.64
7             2817          2519.72         10.55
8             3890          2826.72         27.33
9             3461          2988.64         13.65
10            2611          2731.24         -4.61
11            3249          2649.16         18.46
12            1921          2524.84        -31.43
13            6007          2797.04         53.44
14            3343          2663.92         20.31
15            2446          2722.28        -11.30
16            2806          2842.00         -1.28
17            1910          2671.92        -39.89
18            1834          2404.76        -31.12
19    

In [27]:
%%pyspark
future_date =  pd.date_range(start='2015-12-1', end='2015-12-5')
future_data = pd.DataFrame({'Date':future_date, 'Accessories_count':0})

future_data

Date  Accessories_count
0 2015-12-01                  0
1 2015-12-02                  0
2 2015-12-03                  0
3 2015-12-04                  0
4 2015-12-05                  0

In [28]:
%%pyspark
future_data['Date'] = pd.to_datetime(future_data['Date']).dt.strftime('%Y-%m-%d')

future_data

Date  Accessories_count
0  2015-12-01                  0
1  2015-12-02                  0
2  2015-12-03                  0
3  2015-12-04                  0
4  2015-12-05                  0

## Making future prediction using model that performs best


In [29]:
%%pyspark
future_value = fitted_model.predict(future_data)
temp_df =  pd.DataFrame({'Accessories_count':future_value})
temp_df['Accessories_count'] = temp_df['Accessories_count'].round(0)
future_data['Accessories_Customer_count'] = temp_df['Accessories_count']
future_data.drop({'Accessories_count'},axis=1,inplace=True)
future_data['Date'] = pd.to_datetime(future_data['Date']).dt.strftime('%Y-%m-%d')

future_data

Date  Accessories_Customer_count
0  2015-12-01                     1959.00
1  2015-12-02                     2007.00
2  2015-12-03                     2171.00
3  2015-12-04                     2221.00
4  2015-12-05                     2214.00

In [30]:
%%pyspark
output = spark.createDataFrame(future_data)

## **Registering Model**


In [31]:
%%pyspark
#register model
#model_name = "my_model_20th"
description = "Forecast Model"
tags = None
model = local_run.register_model(description = description, tags = tags)
local_run.model_id

'AutoML07e5388cf6'

In [32]:
%%pyspark
#saving scoring and conda file
script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'
#/content/azureml_automl.log
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/env.yml')

## Checking the progress of the experiment in Azure portal
<p style="font-size:16px; color:#117d30;">Here the URL is retrieved by the following command.</p>



In [33]:
%%pyspark
print(local_run.get_portal_url())

https://ml.azure.com/experiments/Department_Visit_Count_20thFeb_3/runs/AutoML_07e5388c-f6d0-4e14-9bfc-68156dc1ff3f?wsid=/subscriptions/fef31e52-6ea1-46c7-9494-2b5901e9af2b/resourcegroups/Synapse-WWI-Lab/workspaces/amlworkspacedaron12

## Consuming REST EndPoint API


In [34]:
%%pyspark
import urllib.request
import json
import os
import ssl
import pprint

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "Inputs": {
          "WebServiceInput0":
          [
              {
                    'Date': "2020-04-20T00:00:00Z",
                    'Accessories_count': "18",
                    'Entertainment_count': "28",
                    'Gaming': "5",
                    'Kids': "14",
                    'Mens': "14",
                    'Phone_and_GPS': "36",
                    'Womens': "30",
              },
          ],
    },
    "GlobalParameters":  {
    }
}

body = str.encode(json.dumps(data))

url = 'http://13.68.210.52:80/api/v1/service/retail-realtime-inference/score'
api_key = 'wRx4zkptjIjKq41jPFmw9UO3LfZ7CLE6' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    #print(result)
    pprint.pprint(json.loads(result))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


{'Results': {'WebServiceOutput0': [{'Accessories_count': 18,
                                    'Date': '2020-04-20 00:00:00',
                                    'Entertainment_count': 28,
                                    'Gaming': 5,
                                    'Kids': 14,
                                    'Mens': 14,
                                    'Phone_and_GPS': 36,
                                    'Scored Labels': 14,
                                    'Scored Probabilities_0': 0.0,
                                    'Scored Probabilities_14': 0.125,
                                    'Scored Probabilities_16': 0.0,
                                    'Scored Probabilities_21': 0.125,
                                    'Scored Probabilities_22': 0.0,
                                    'Scored Probabilities_23': 0.125,
                                    'Scored Probabilities_25': 0.125,
                                    'Scored Probabilities_29': 0.0,